<h3>COVID Tracking in my County:</h3>
The script processes the covid infections tally from the state and converts it to excel used to have an extra data point of the most current level of infection in our county and the county of the university of our kids.

In [ ]:
import pandas as pd    
import numpy as np
import scipy.stats as stats
import re
import datetime
from datetime import datetime, date, time, timedelta
#
#
# =================
# == ROUTINES
# =================
#
def mk_dt (x, f): #$03-07-2020 Test: 2020-09-13
    import datetime
    str_f = "%m/%d/%Y"
    #
    if (not isinstance(x, datetime.datetime) and not re.match('County', x)):
        x = datetime.datetime.strptime(x, str_f)
    #./.
    #
    return(x)

def mr_excel (fname, skipr, fty, myear, mrcty):
    #
    import datetime
    #
    if (fty == "casen" or fty == "case"):
        sheetn = "Cases by County " + myear
    if (fty == "dead"):
        sheetn = "Fatalities by County " + myear
    if (fty == "test"):
        sheetn = "Total Tests " + myear        
    #
    xls0  = pd.ExcelFile(fname)
    df = pd.read_excel(xls0, sheet_name=sheetn, skiprows= skipr)
    county = 'County'
    if (fty == "case"): #-- 01/18 or fty == "dead"):
        county = 'County Name'  
        txtm   = 'Cases '
    #./.
    #
    df[county] = df[county].str.lower()
    df = df[df[county] == mrcty]    
    del df[county]
    #
    if (fty == "case"): df.rename(columns=lambda x : re.sub(txtm,'',x), inplace=True)    
    df.rename(columns=lambda x : mk_dt(x, fty), inplace=True)
    #
    last_t = 0    
    for c in list(df.columns):
        if isinstance(c, datetime.datetime):            
            new_t  = df[c].iloc[0] - last_t
            last_t = df[c].iloc[0]
            df[c].iloc[0] = new_t
        #./. if        
    #./. for
    return(df)

In [ ]:
#
# =================
# == MAIN CODE
# =================
#
#
#YEAR_SCREEN = 2022 county= All lowercase
def county_data (county, YEAR_SCREEN):
    #
    print("\n=======================================================")
    print("=== LOADING FILES FOR COUNTY: ", county, " YEAR: ", YEAR_SCREEN," ===")
    print("=======================================================")
    # From Website: https://www.arcgis.com/apps/dashboards/45e18cba105c478697c76acbbf86a6bc
    case_df = mr_excel ("./Texas COVID-19 Cumulative Confirmed Cases by County.xlsx", 2, "casen", str(YEAR_SCREEN), county)
    test_df = mr_excel ("./Cumulative Tests over Time by County.xlsx", 1, "test", str(YEAR_SCREEN), county)
    dead_df = mr_excel ("./Texas COVID-19 Fatality Count Data by County.xlsx", 2, "dead", str(YEAR_SCREEN), county)
    #
    print("\n=== Creating dataFrame where data exist per date")
    i=0
    dirc = {}
    for c in list(test_df.columns): # 375 common entries
        if (c in list(dead_df.columns) and c in list(case_df.columns)):        
            case = case_df[c].iloc[0]
            dead = dead_df[c].iloc[0]
            test = test_df[c].iloc[0]
            if (test == 0):
                prate = np.nan
            else:
                prate = case/test
            #./.if
            #
            if i != 0: 
                dirc.update({c:[case, test, dead, prate]})
            #./.
            #
            i += 1
    #./. for
    #
    #test_df
    #dirc
    print("\t* Transposing Dframe so 'date' is the index")
    out_df = pd.DataFrame(dirc, index=["case", "test", "dead", "prate"])
    out_dft = out_df.T
    out_dft.dropna(inplace=True)
    #
    # Now group by month and do math. Spit out excel just to graph
    #
    print("\n=== Grouping per month, create DF")
    month_grp = out_dft.groupby(by=[out_dft.index.month, out_dft.index.year])
    case_ser = month_grp['case'].sum()
    test_ser = month_grp['test'].sum()
    dead_ser = month_grp['dead'].sum()
    #
    # Create per month Df
    month_df = pd.DataFrame([case_ser, test_ser, dead_ser])
    #
    print("\t* Transpose so date/year is index. Add +ve rate")
    m_dft = month_df.T
    m_dft.index.rename(["date","year"], inplace=True)
    m_dft = m_dft.sort_values(["year","date"])
    m_dft['prate'] = m_dft['case']/m_dft.test
    #
    print("\t* Select data for current year:", YEAR_SCREEN)
    m_dft.reset_index('year', inplace=True)
    m_dft = m_dft[m_dft['year'] == YEAR_SCREEN]
    del m_dft['year']
    #
    print("\t* Creating excel file")
    file_out = "./Output/out_mr_"+county+".xlsx"
    m_dft.to_excel(file_out, sheet_name='Sheet')#, index=False) 
    print("./. Done")
    #
    curr_prate = m_dft.prate.iloc[-1]
    curr_prate *= 100
    if (curr_prate <= 5):
        print("\n+++ Positivty rate below WHO 5%. Current:", np.round_(curr_prate, decimals = 3),"%")
        print("    It needs to be 14 days at this level")          
    else:
        print("\n--- Positivty rate above WHO 5%. Current:", np.round_(curr_prate, decimals = 3),"%")
    #./.
    #
    total_popu  = 1095580    
    #
    # 2020, 2021 data below confirmed with Texas Health site (CaseCount.xlsx)
    dead_pre22  = 1223
    cases_pre22 = 114653
    #    
    if (county == 'county1'):
        total_popu  = 944139
        #
        # 2020, 2021 data below confirmed with Texas Health site (CaseCount.xlsx)
        dead_pre22  = 405 + 692
        cases_pre22 = 35158 + 52980
    #./. eo if
    #    
    total_dead = dead_pre22  + m_dft['dead'].sum()
    total_case = cases_pre22 + m_dft['case'].sum()
    #
    dead_p = total_dead/total_popu * 100
    case_p = total_case/total_popu * 100
    #
    print ("\n ",county,"county population 2021: ",round(total_popu))
    print ("\t% Dead     of COVID (2020-22): {0:6.0f} {1:5.2f}%".format(total_dead, dead_p))
    print ("\t% Infected of COVID (2020-22): {0:6.0f} {1:5.2f}%".format(total_case, case_p))
    #
    dead_flu = 1000000/(28000000/3500)
    print ("\n      circa ",round(dead_flu)," Flu deaths in all of 2018")
    #
#./. eo def

YEAR_SCREEN = 2022
county = 'County1' # I: All lowercase
county_data (county, YEAR_SCREEN)
#
county = 'County2' # I: All lowercase
county_data (county, YEAR_SCREEN)
